In [6]:
import numpy as np
np.random.seed(1338)

from keras.datasets import mnist

from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Dropout, Activation
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD


(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_test_orig = x_test
from keras import backend as k

img_rows, img_cols = x_train.shape[1:]
if k.image_data_format == 'channels_first':
    shape_ord = (1, img_rows, img_cols)
else:
    shape_ord = (img_rows, img_cols, 1)
    
    
x_train = x_train.reshape((x_train.shape[0],) + shape_ord)
x_test = x_test.reshape((x_test.shape[0],) + shape_ord)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255.0
x_test /= 255.0


y_train =np_utils.to_categorical(y_train, 10) 
y_test = np_utils.to_categorical(y_test, 10)


from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train)


In [7]:
import matplotlib.pyplot as plt
%matplotlib inline
import time
from tensorflow.keras.callbacks import TensorBoard
epochs = 5
batch_size = 64
filter_size = 3
pool_size = 2

conv_layers = 6
dense_layers = 3

NAME="{}-conv-{}-dense-{}".format(conv_layers, dense_layers, int(time.time()))
tensorboard = TensorBoard(log_dir='CNN_ICP_5/{}'.format(NAME))

sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)


model = Sequential()
model.add(Conv2D(32, (filter_size, filter_size), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv2D(32, (filter_size, filter_size), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Conv2D(64, (filter_size, filter_size), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv2D(64, (filter_size, filter_size), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Conv2D(128, (filter_size, filter_size), padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (filter_size, filter_size), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val), verbose=1, callbacks=[tensorboard])



Train on 45000 samples, validate on 15000 samples
Epoch 1/5
45000/45000 [==============================] - 387s 9ms/step - loss: 2.3070 - acc: 0.1038 - val_loss: 2.3018 - val_acc: 0.0995
Epoch 2/5
45000/45000 [==============================] - 389s 9ms/step - loss: 2.3048 - acc: 0.1070 - val_loss: 2.3027 - val_acc: 0.1152
Epoch 3/5
45000/45000 [==============================] - 398s 9ms/step - loss: 2.3048 - acc: 0.1043 - val_loss: 2.3026 - val_acc: 0.0975
Epoch 4/5
45000/45000 [==============================] - 430s 10ms/step - loss: 2.3048 - acc: 0.1043 - val_loss: 2.3061 - val_acc: 0.1152
Epoch 5/5
45000/45000 [==============================] - 388s 9ms/step - loss: 2.3046 - acc: 0.1042 - val_loss: 2.3031 - val_acc: 0.0985


In [8]:
loss, accuracy =model.evaluate(x_test, y_test, verbose=0)
print(loss)
print(accuracy)


2.3032746505737305
0.0974


In [17]:
slice = 5

predicted = model.predict(x_test[:slice]).argmax(-1)
actual = model.predict_classes(x_test[:slice], verbose=1)
print(actual)

print(predicted)

5/5 [==============================] - 0s 2ms/step
[8 8 8 8 8]
[8 8 8 8 8]


NameError: name 'decode_predictions' is not defined